# **Generate data and install Mosek**

Please install Mosek according to the instructions on [https://docs.mosek.com/latest/install/installation.html](https://docs.mosek.com/latest/install/installation.html).

Before running this cell, run the following in bash - 

`export PATH=/root/mosek/9.3/tools/platform/linux64x86/bin:$PATH`.

Please download MOSEK from [here](https://drive.google.com/drive/folders/1ZIKizbByyQIZmWCPTgFvnnQ6bVzUcRvF?usp=sharing) and add all contents to the current working directory. 

Please replace *mosek.lic* with your own license file obtained from [here](https://www.mosek.com/products/academic-licenses/).

In [ ]:
! bash install-mosek.sh
! bash create-leopard-environment.sh

# **LEOPARD**

Setup the environment required for the classfications which are defined by Bansal et al. 

In [ ]:
import os
import json

DATA_PATH = "/content/leopard/data/json"

def get_categories():
    categories = []
    categories.append("restaurant")
    categories.append("conll")
    return categories

def get_labelled_training_sentences(category, shot, episode):
    sentences = []
    labels = []
    entities = []
    label_keys = {}
    label_index = 0
    data_path = DATA_PATH + "/" + category + "/"
    for file_name in os.listdir(data_path):
        if file_name.endswith("_" + str(episode) + "_" + str(shot) + ".json"):
            data = json.load(open(data_path + file_name))            
            for index in range(len(data)):
                sentence_1 = data[index]['sentence1']
                entity = data[index]['sentence2']
                label = data[index]['label']
                sentences.append(sentence_1)
                entities.append(entity)
                # convert categorical labels to numeric values
                if label not in label_keys:
                    label_keys[label] = label_index
                    label_index += 1
                labels.append(label_keys[label])
    return sentences, entities, labels, label_keys

def get_labelled_test_sentences(category):
    sentences = []
    labels = []
    entities = []
    data_path = DATA_PATH + "/" + category + "/"
    for file_name in os.listdir(data_path):
        if file_name.endswith("_eval.json"):
            data = json.load(open(data_path + file_name))            
            for index in range(len(data)):
                sentence_1 = data[index]['sentence1']
                entity = data[index]['sentence2']
                label = data[index]['label']
                sentences.append(sentence_1)
                labels.append(label)
                entities.append(entity)
    
    return sentences, entities, labels

# **Less Than One Shot Classification (Sucholutsky et al, 2021)**

Original source can be found [here](https://github.com/ilia10000/LO-Shot/blob/master/Paper3/Soft%20Label%20Optimization.ipynb). Run the script to install the environment.

In [ ]:
! bash create-lo-shot-environment.sh

Load the required scripts and definitions.

In [ ]:
% run lo_shot_definitions.ipynb

# **Extracting the data and creating the training set**


Define the encoder.

In [ ]:
from transformers import BertTokenizer, BertModel
import torch
import traceback

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

Generate the sentence encoding for each sentence. Store this along with the labels; use it to generate lines and prototypes.

In [ ]:
import numpy as np

def get_labelled_training_data(category, shot, episode):
    sentences, entities, training_labels, label_keys = get_labelled_training_sentences(category, shot, episode)

    training_encodings = []
    for sentence, entity, label in zip(sentences, entities, training_labels):
        inputs = tokenizer(entity, return_tensors="pt")
        required_bert_tokens = [e for e in inputs['input_ids'].tolist()[0] if e not in [101, 102]]
        inputs = tokenizer(sentence, return_tensors="pt")
        outputs = model(**inputs)
        
        inputs_indices_sentence = []
        for i in range(len(inputs['input_ids'].tolist()[0])):
            if inputs['input_ids'].tolist()[0][i] in required_bert_tokens:
                required_bert_tokens.remove(inputs['input_ids'].tolist()[0][i])
                inputs_indices_sentence.append(i)
        encodings = []
        for inputs_index_sentence in inputs_indices_sentence:
            encodings.append(outputs.last_hidden_state[:,inputs_index_sentence,:].detach().numpy())
        encoding = np.stack(encodings).mean(axis=0).reshape(-1)
        training_encodings.append(encoding)

    return training_encodings, training_labels, label_keys

Get centroids for the training data.

In [ ]:
def get_labelled_centroids(training_encodings, training_labels):
    centroids = []
    centroid_labels = []
    for label in set(training_labels):
        centroids_per_label = []
        for i in range(len(training_labels)):
            if training_labels[i] == label:
                centroids_per_label.append(training_encodings[i])
        centroid = np.mean(np.array(centroids_per_label), axis=0)
        centroids.append(centroid)
        centroid_labels.append(label)
    
    return centroids, centroid_labels

Create the test dataset for the sentences.

In [ ]:
def get_labelled_test_data(category):
    sentences, entities, test_labels = get_labelled_test_sentences(category)

    test_encodings = []
    for sentence, entity, label in zip(sentences, entities, test_labels):
        inputs = tokenizer(entity, return_tensors="pt")
        required_bert_tokens = [e for e in inputs['input_ids'].tolist()[0] if e not in [101, 102]]
        inputs = tokenizer(sentence, return_tensors="pt")
        outputs = model(**inputs)
        
        inputs_indices_sentence = []
        for i in range(len(inputs['input_ids'].tolist()[0])):
            if inputs['input_ids'].tolist()[0][i] in required_bert_tokens:
                required_bert_tokens.remove(inputs['input_ids'].tolist()[0][i])
                inputs_indices_sentence.append(i)
        encodings = []
        for inputs_index_sentence in inputs_indices_sentence:
            encodings.append(outputs.last_hidden_state[:,inputs_index_sentence,:].detach().numpy())
        encoding = np.stack(encodings).mean(axis=0).reshape(-1)
        test_encodings.append(encoding)

    return test_encodings, test_labels

# **Classification using different flavours of KNN**
Perform classification on the data using vanilla KNN.

In [ ]:
def classify_with_vanilla_KNN(test_classifications, labeled_centroids, labeled_test_data):
  
  classifier_KNN = SoftKNN(k=1)
  knn_labels = np.zeros((test_classifications, test_classifications))
  np.fill_diagonal(knn_labels, 1)
  classifier_KNN.fit(labeled_centroids[0], knn_labels)
  
  vanilla_knn_preds = classifier_KNN.predict(labeled_test_data[0])
  vanilla_knn_correct = np.sum(vanilla_knn_preds==labeled_test_data[1])
  
  return vanilla_knn_correct, len(vanilla_knn_preds)

Perform the classification using soft label KNN

In [ ]:
def get_soft_label_prototypes(lines, labeled_centroids):

  classifiers = []
  for line in lines:
    try:
      distX, distY = get_line_prototypes(line, labeled_centroids[0])
      classifier = SoftKNN(k=1)
      classifier.fit(distX, distY)
      classifiers.append(classifier)
    except:
      classifiers.append(None)

  return classifiers

# **Classify and measure metrics**

Classify the test data using centroids from the training data.

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

assert(cp.MOSEK in cp.installed_solvers())

for category in get_categories():
    test_encodings, categorical_test_labels = get_labelled_test_data(category)
    for episode in range(10):
        for shot in [4,8,16]:
            predictions = []
            true_labels = []

            training_encodings, training_labels, label_keys = get_labelled_training_data(category, shot, episode)            
            if not training_encodings:
                continue
            # convert categorical attributes to numeric indices
            test_labels = [label_keys[label] for label in categorical_test_labels]        
            centroids, centroid_labels = get_labelled_centroids(training_encodings, training_labels)

            labeled_training_data_np = np.stack(training_encodings, axis=0)
            labeled_test_data_np = np.stack(test_encodings, axis=0)
            labeled_centroids_np = np.stack(centroids, axis=0)

            labeled_test_data = [labeled_test_data_np[i] for i in range(labeled_test_data_np.shape[0])], [label for label in test_labels]
            labeled_centroids = [labeled_centroids_np[i] for i in range(labeled_centroids_np.shape[0])], [label for label in centroid_labels]

            test_classifications = max(max(labeled_centroids[1]) for labeled_centroid in labeled_centroids) + 1
            required_lines = test_classifications - 1
            dimensions = labeled_training_data_np.shape[1]

            lines = [line_order_no_endpoints(centroids=labeled_centroids_np, active_classes=np.array(line)) for line in find_lines_R_multiD(dat=labeled_training_data_np, labels=training_labels , dims=dimensions, centroids=labeled_centroids_np, k=required_lines)]
            classifiers = get_soft_label_prototypes(lines, labeled_centroids)

            for i in range(test_classifications):
                points = int(np.sum([True for class_val in labeled_test_data[1] if class_val == i]))
                points_required = [labeled_test_data[0][x] for x in range(len(labeled_test_data[1])) if labeled_test_data[1][x] == i]
                assignments = []
                for point in points_required:
                    dists = [dist_to_line_multiD(point, labeled_centroids[0][line[0]], labeled_centroids[0][line[-1]]) for line in lines]
                    nearest = np.argmin(dists)
                    assignments.append(nearest)
                for j in range(len(points_required)):
                    if classifiers[assignments[j]] is not None:
                        classifier = classifiers[assignments[j]]
                        prediction = classifier.predict(points_required[j])
                        predictions.append(prediction)
                        true_labels.append(i)

            print("For category", category, "and shot =", str(shot) + "...")
            print("Lines used are", len(lines))
            print("Number of classifications are", test_classifications)
            print("Macro f1 score is", f1_score(true_labels, predictions, average='macro'))
            print("Accuracy is", accuracy_score(true_labels, predictions))
            print("Correctly classified points are", np.sum(np.array(true_labels) == np.array(predictions)), "/", len(true_labels), "\n")